In [1]:
import pandas as pd
import numpy as np
from datetime import date

# combined data - 2 tables

In [2]:
path_tables = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 2 tables/"

In [3]:
filename_careers = "careers.xlsx"
careers = pd.read_excel(path_tables + filename_careers,dtype="str")
careers.shape

(9001, 12)

In [4]:
filename_orgtree = "orgtree.xlsx"
ot = pd.read_excel(path_tables + filename_orgtree,dtype="str")
ot.shape

(1765, 9)

In [5]:
filename_leadercareerlink = "leadercareerlink.xlsx"
lclink = pd.read_excel(path_tables + filename_leadercareerlink,dtype="str")
lclink.shape

(12617, 3)

In [6]:
# filename_leaderjoblink = "leaderjoblink.xlsx"

In [7]:
# filename_joborglink = "joborglink.xlsx"

# combined data - 3 queries

In [8]:
path_queries = "C:/Users/seoul/Dropbox/00 technical/github/nkelites/data/combined data/combined data - 3 queries/"

In [9]:
filename_leaderjobtransition = "leaderjobtransition.xlsx"
# ljt = pd.read_excel(path_queries + filename_leaderjobtransition,dtype="str")

In [10]:
today = date.today()
print(today)

2023-07-13


# functions

In [11]:
def merge_results(m):
    
    print("\nMerge Results...")
    print("")
    print("\tshape     :",m.shape)
    print("\tleft_only :",m[m["_merge"]=="left_only"].shape)
    print("\tboth      :",m[m["_merge"]=="both"].shape)
    print("\tright_only:",m[m["_merge"]=="right_only"].shape)

# 1. deprecate old version

In [12]:
table_name = "leaderjobtransition"
table_name

'leaderjobtransition'

In [13]:
filename_leaderjobtransition

'leaderjobtransition.xlsx'

In [14]:
filename_deprecated = table_name + "_deprecated_"+str(today)+".xlsx"
filename_deprecated

'leaderjobtransition_deprecated_2023-07-13.xlsx'

In [15]:
# deprecate old query
# oldtable = pd.read_excel(path_queries + filename_careers,dtype="str")
# oldtable.to_excel(path_queries + filename_deprecated,index=False)

# 2. refresh query using current tables

In [32]:
lclink.head(2)

,LeaderID,CareerString,CareerDateString_2022
0,리선권,개성공단 남북공동위원회 통행통신통관 분과위원회,2013.09 ~ 2014.01
1,조경철,故 김정일 국가장의위원회 위원,2011.12


In [33]:
# get jobs from careers
jobs = careers[(careers["IsJob"]=="True") & (careers["CareerStartYear"].notna())]

In [117]:
# leader-job query - left join - includes IsJob True & False
key_columns = ["CareerString","CareerDateString_2022"]
lc = lclink.merge(jobs,on=key_columns,how="left",indicator=True)
lc.shape

(13583, 14)

In [118]:
merge_results(lc)


Merge Results...

	shape     : (13583, 14)
	left_only : (4591, 14)
	both      : (8992, 14)
	right_only: (0, 14)


In [119]:
# no "left-only" cases when IsJob=True
lc[(lc["IsJob"]=="True") & (lc["_merge"]=="left_only")]

,LeaderID,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes,_merge


In [120]:
# leader-job query - inner join - only includes IsJob = True
key_columns = ["CareerString","CareerDateString_2022"]
lc = lclink.merge(jobs,on=key_columns,how="inner",indicator=False)
lc = lc.sort_values("LeaderID",ignore_index=True)
lc.shape

(8992, 13)

In [121]:
# instantiate CareerStartDate
lc["CareerStartDate"]=np.nan
lc.loc[lc["CareerStartMonth"].isna(),"CareerStartDate"] = lc[lc["CareerStartMonth"].isna()].apply(lambda x: str(x["CareerStartYear"]) + "00",axis=1)
lc.loc[(lc["CareerStartMonth"].notna()) & (lc.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==2),"CareerStartDate"] = lc[(lc["CareerStartMonth"].notna()) & (lc.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==2)].apply(lambda x: str(x["CareerStartYear"]) + str(x["CareerStartMonth"]),axis=1)
lc.loc[(lc["CareerStartMonth"].notna()) & (lc.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==1),"CareerStartDate"] = lc[(lc["CareerStartMonth"].notna()) & (lc.apply(lambda x: len(str(x["CareerStartMonth"])),axis=1)==1)].apply(lambda x: str(x["CareerStartYear"]) + "0" + str(x["CareerStartMonth"]),axis=1)

In [122]:
lc.shape

(8992, 14)

In [125]:
pd.set_option('display.max_rows', None)

In [127]:
choigo = lc[lc["PrimaryInstitution"]=="최고인민회의"]
po_columns = ["PrimaryInstitution","OrgName","Position"]
choigo[po_columns].drop_duplicates(po_columns,keep="first").sort_values(po_columns)

,PrimaryInstitution,OrgName,Position
2001,최고인민회의,강원도인민위원회,부위원장
504,최고인민회의,강원도인민위원회,위원장
3388,최고인민회의,강원도인민위원회_문천군인민위원회,위원장
1995,최고인민회의,강원도인민위원회_원산시인민위원회,위원장
600,최고인민회의,개성시인민위원회,부위원장
2306,최고인민회의,개성시인민위원회,위원장
1401,최고인민회의,나선시인민위원회,위원장
3596,최고인민회의,남포시인민위원회,부위원장
3594,최고인민회의,남포시인민위원회,위원장
4472,최고인민회의,남포시인민위원회_천리마구역인민위원회,위원장


In [113]:
# remove 최고인민회의
# lc = lc[~((lc["PrimaryInstitution"]=="최고인민회의") & (lc["OrgName"].isna()))]
lc.shape

(6960, 14)

In [114]:
# remove 노동당, 당중앙위원회
lc = lc[~((lc["PrimaryInstitution"]=="노동당") & (lc["OrgName"]=="당중앙위원회"))]
lc.shape

(5967, 14)

In [115]:
t = lc[(lc["LeaderID"]=="강관주") & (lc["CareerStartYear"].notna())]
t = t.drop_duplicates(["CareerStartYear","PrimaryInstitution","OrgName"],keep="first")
t = t.sort_values("CareerStartDate",ignore_index=True)
t.shape

(17, 14)

In [116]:
t

,LeaderID,CareerString,CareerDateString_2022,IsJob,MultipleSubstrings,CareerStartYear,CareerStartMonth,CareerSubstring,OrgString,PrimaryInstitution,OrgName,Position,Notes,CareerStartDate
0,강관주,1961. 체육과학연구소 연구원,NaN,True,1,1961,NaN,NaN,체육과학연구소,체육연구원,체육과학연구소,연구원,NaN,196100
1,강관주,만경대예술단 섭외부 부부장,1973,True,1,1973,NaN,NaN,만경대예술단 섭외부,평양만경대예술단,섭외부,부부장,NaN,197300
2,강관주,당 문화부 과장,1973,True,1,1973,NaN,NaN,당 문화부,노동당,문화예술부,과장,NaN,197300
3,강관주,대외문화연락협회 국장,1975.05,True,1,1975,5,NaN,대외문화연락협회,조선대외문화연락협회,NaN,국장,NaN,197505
4,강관주,1986. 당 조직지도부 부부장,NaN,True,1,1986,NaN,NaN,노동당 조직지도부,노동당,당중앙위원회_조직지도부,부부장,NaN,198600
5,강관주,1988. 당 통일전선부 #조총련# 담당 부부장,NaN,True,1,1988,NaN,NaN,노동당 중앙위원회 통일전선부,노동당,당중앙위원회_통일전선부_문화교류국,부부장,NaN,198800
6,강관주,조선노동당 통일전선부 조선인총연합회 담당 부부장,1988,True,1,1988,NaN,NaN,조선노동당 통일전선부,노동당,당중앙위원회_통일전선부,부부장,NaN,198800
7,강관주,조선노동당 통일전선부 제1부부장,1989.03,True,1,1989,3,NaN,조선노동당 통일전선부 제1부,노동당,당중앙위원회_통일전선부,부장,NaN,198903
8,강관주,조국평화통일위원회 부위원장,1990.08,True,1,1990,8,NaN,조국통일평화위원회,정무원,조국평화통일위원회,부위원장,NaN,199008
9,강관주,1990. 8 조국평화통일위원회 부위원장,NaN,True,1,1990,8,NaN,조국평화통일위원회,내각,조국평화통일위원회,부위원장,NaN,199008


In [97]:
tr = t.merge(t,on="LeaderID",how="inner",suffixes=("_1","_2"))

In [98]:
# sort based on CareerStartDates
tr = tr.sort_values(["CareerStartDate_1","CareerStartDate_2"])
tr.shape

(484, 27)

In [99]:
# drop keys where CareerStartDate_1 <= CareerStartDate_2
tr = tr[tr.apply(lambda x: x["CareerStartDate_1"] < x["CareerStartDate_2"],axis=1)]
tr.shape

(226, 27)

In [100]:
# smallest next startdate
dates = tr[["CareerStartDate_1","CareerStartDate_2"]]

In [101]:
dates

,CareerStartDate_1,CareerStartDate_2
1,196100,197300
2,196100,197300
3,196100,197505
4,196100,198600
5,196100,198800
...,...,...
416,199809,200900
417,199809,201009
438,200309,200900
439,200309,201009


In [102]:
mindate = dates.groupby("CareerStartDate_1",as_index=False).agg({"CareerStartDate_2":"min"})
mindate.shape

(16, 2)

In [104]:
mindate

,CareerStartDate_1,CareerStartDate_2
0,196100,197300
1,197300,197505
2,197505,198600
3,198600,198800
4,198800,198803
5,198803,198903
6,198903,199004
7,199004,199008
8,199008,199109
9,199109,199301


In [105]:
mindate.columns = ["CareerStartDate_1","CareerStartDate_2_min"]

In [106]:
tr2 = tr.merge(mindate,on="CareerStartDate_1",how="left")
tr2.shape

(226, 28)

In [107]:
tr3 = tr2[tr2.apply(lambda x: x["CareerStartDate_2"] == x["CareerStartDate_2_min"],axis=1)]
tr3.shape

(26, 28)

In [108]:
tr3

,LeaderID,CareerString_1,CareerDateString_2022_1,IsJob_1,MultipleSubstrings_1,CareerStartYear_1,CareerStartMonth_1,CareerSubstring_1,OrgString_1,PrimaryInstitution_1,...,CareerStartYear_2,CareerStartMonth_2,CareerSubstring_2,OrgString_2,PrimaryInstitution_2,OrgName_2,Position_2,Notes_2,CareerStartDate_2,CareerStartDate_2_min
0,강관주,1961. 체육과학연구소 연구원,NaN,True,1,1961,NaN,NaN,체육과학연구소,체육연구원,...,1973,NaN,NaN,만경대예술단 섭외부,평양만경대예술단,섭외부,부부장,NaN,197300,197300
1,강관주,1961. 체육과학연구소 연구원,NaN,True,1,1961,NaN,NaN,체육과학연구소,체육연구원,...,1973,NaN,NaN,당 문화부,노동당,문화예술부,과장,NaN,197300,197300
21,강관주,만경대예술단 섭외부 부부장,1973,True,1,1973,NaN,NaN,만경대예술단 섭외부,평양만경대예술단,...,1975,5,NaN,대외문화연락협회,조선대외문화연락협회,NaN,국장,NaN,197505,197505
22,강관주,당 문화부 과장,1973,True,1,1973,NaN,NaN,당 문화부,노동당,...,1975,5,NaN,대외문화연락협회,조선대외문화연락협회,NaN,국장,NaN,197505,197505
59,강관주,대외문화연락협회 국장,1975.05,True,1,1975,5,NaN,대외문화연락협회,조선대외문화연락협회,...,1986,NaN,NaN,노동당 조직지도부,노동당,당중앙위원회_조직지도부,부부장,NaN,198600,198600
77,강관주,1986. 당 조직지도부 부부장,NaN,True,1,1986,NaN,NaN,노동당 조직지도부,노동당,...,1988,NaN,NaN,노동당 중앙위원회 통일전선부,노동당,당중앙위원회_통일전선부_문화교류국,부부장,NaN,198800,198800
78,강관주,1986. 당 조직지도부 부부장,NaN,True,1,1986,NaN,NaN,노동당 조직지도부,노동당,...,1988,NaN,NaN,조선노동당 통일전선부,노동당,당중앙위원회_통일전선부,부부장,NaN,198800,198800
94,강관주,1988. 당 통일전선부 #조총련# 담당 부부장,NaN,True,1,1988,NaN,NaN,노동당 중앙위원회 통일전선부,노동당,...,1988,3,NaN,조선노동당 중앙위원회,노동당,당중앙위원회,위원,NaN,198803,198803
95,강관주,조선노동당 통일전선부 조선인총연합회 담당 부부장,1988,True,1,1988,NaN,NaN,조선노동당 통일전선부,노동당,...,1988,3,NaN,조선노동당 중앙위원회,노동당,당중앙위원회,위원,NaN,198803,198803
124,강관주,조선노동당 중앙위원회 위원,1988.03,True,1,1988,3,NaN,조선노동당 중앙위원회,노동당,...,1989,3,NaN,조선노동당 통일전선부 제1부,노동당,당중앙위원회_통일전선부,부장,NaN,198903,198903


In [110]:
some_columns = ["LeaderID","CareerStartDate_1","PrimaryInstitution_1","OrgName_1","Position_1","CareerStartDate_2","PrimaryInstitution_2","OrgName_2","Position_2"]
tr3[some_columns]

,LeaderID,CareerStartDate_1,PrimaryInstitution_1,OrgName_1,Position_1,CareerStartDate_2,PrimaryInstitution_2,OrgName_2,Position_2
0,강관주,196100,체육연구원,체육과학연구소,연구원,197300,평양만경대예술단,섭외부,부부장
1,강관주,196100,체육연구원,체육과학연구소,연구원,197300,노동당,문화예술부,과장
21,강관주,197300,평양만경대예술단,섭외부,부부장,197505,조선대외문화연락협회,NaN,국장
22,강관주,197300,노동당,문화예술부,과장,197505,조선대외문화연락협회,NaN,국장
59,강관주,197505,조선대외문화연락협회,NaN,국장,198600,노동당,당중앙위원회_조직지도부,부부장
77,강관주,198600,노동당,당중앙위원회_조직지도부,부부장,198800,노동당,당중앙위원회_통일전선부_문화교류국,부부장
78,강관주,198600,노동당,당중앙위원회_조직지도부,부부장,198800,노동당,당중앙위원회_통일전선부,부부장
94,강관주,198800,노동당,당중앙위원회_통일전선부_문화교류국,부부장,198803,노동당,당중앙위원회,위원
95,강관주,198800,노동당,당중앙위원회_통일전선부,부부장,198803,노동당,당중앙위원회,위원
124,강관주,198803,노동당,당중앙위원회,위원,198903,노동당,당중앙위원회_통일전선부,부장


# 3. export new query

In [55]:
# refresh leaderjobtransition with new data
ljt_new.to_excel(path_tables + filename_careers,index=False)